In [8]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp

In [3]:
class OrbitCamera:
    def __init__(self, W, H, r=2, fovy=60):
        self.W = W
        self.H = H
        self.radius = r # camera distance from center
        self.fovy = fovy # in degree
        self.center = np.array([0, 0, 0], dtype=np.float32) # look at this point
        self.rot = R.from_quat([1, 0, 0, 0]) # init camera matrix: [[1, 0, 0], [0, -1, 0], [0, 0, 1]] (to suit ngp convention)
        self.up = np.array([0, 1, 0], dtype=np.float32) # need to be normalized!

    # pose
    @property
    def pose(self):
        # first move camera to radius
        res = np.eye(4, dtype=np.float32)
        res[2, 3] -= self.radius
        # rotate
        rot = np.eye(4, dtype=np.float32)
        rot[:3, :3] = self.rot.as_matrix()
        res = rot @ res
        # translate
        res[:3, 3] -= self.center
        return res
    
    # intrinsics
    @property
    def intrinsics(self):
        focal = self.H / (2 * np.tan(np.radians(self.fovy) / 2))
        return np.array([focal, focal, self.W // 2, self.H // 2])
    
    def orbit(self, dx, dy):
        # rotate along camera up/side axis!
        side = self.rot.as_matrix()[:3, 0] # why this is side --> ? # already normalized.
        rotvec_x = self.up * np.radians(-0.1 * dx)
        rotvec_y = side * np.radians(-0.1 * dy)
        self.rot = R.from_rotvec(rotvec_x) * R.from_rotvec(rotvec_y) * self.rot

    def scale(self, delta):
        self.radius *= 1.1 ** (-delta)

    def pan(self, dx, dy, dz=0):
        # pan in camera coordinate system (careful on the sensitivity!)
        self.center += 0.0005 * self.rot.as_matrix()[:3, :3] @ np.array([dx, dy, dz])


In [45]:
import numpy as np

N = 300  # Number of points

# Create evenly spaced grid along x-axis
x = np.linspace(-50, 50, N)
x, y = np.meshgrid(x, x)

# Reshape grid into a 2D array
grid = np.column_stack((x.flatten(), y.flatten()))

# Adjust the number of points to N
grid = grid[:N]

# Print the resulting grid
print(grid.shape)
print(grid)

(300, 2)
[[-50.         -50.        ]
 [-49.66555184 -50.        ]
 [-49.33110368 -50.        ]
 [-48.99665552 -50.        ]
 [-48.66220736 -50.        ]
 [-48.3277592  -50.        ]
 [-47.99331104 -50.        ]
 [-47.65886288 -50.        ]
 [-47.32441472 -50.        ]
 [-46.98996656 -50.        ]
 [-46.65551839 -50.        ]
 [-46.32107023 -50.        ]
 [-45.98662207 -50.        ]
 [-45.65217391 -50.        ]
 [-45.31772575 -50.        ]
 [-44.98327759 -50.        ]
 [-44.64882943 -50.        ]
 [-44.31438127 -50.        ]
 [-43.97993311 -50.        ]
 [-43.64548495 -50.        ]
 [-43.31103679 -50.        ]
 [-42.97658863 -50.        ]
 [-42.64214047 -50.        ]
 [-42.30769231 -50.        ]
 [-41.97324415 -50.        ]
 [-41.63879599 -50.        ]
 [-41.30434783 -50.        ]
 [-40.96989967 -50.        ]
 [-40.63545151 -50.        ]
 [-40.30100334 -50.        ]
 [-39.96655518 -50.        ]
 [-39.63210702 -50.        ]
 [-39.29765886 -50.        ]
 [-38.9632107  -50.        ]
 [-38

In [ ]:
# use the dx and dy generated above to create the camera poses for rendering ....

In [34]:
from numpy import random

def random_radius(min_radius=1.0, max_radius=5.0, seed = 0):
    random.seed(seed)
    return np.random.uniform(min_radius, max_radius)

def random_quaternion(seed = 0):
    random.seed(seed)
    theta = np.random.uniform(0, 2 * np.pi)
    phi = np.random.uniform(0, np.pi)
    psi = np.random.uniform(0, 2 * np.pi)

    qx = np.sin(phi / 2) * np.cos(theta / 2) * np.cos(psi / 2) - np.cos(phi / 2) * np.sin(theta / 2) * np.sin(psi / 2)
    qy = np.sin(phi / 2) * np.cos(theta / 2) * np.sin(psi / 2) + np.cos(phi / 2) * np.sin(theta / 2) * np.cos(psi / 2)
    qz = np.cos(phi / 2) * np.cos(theta / 2) * np.sin(psi / 2) - np.sin(phi / 2) * np.sin(theta / 2) * np.cos(psi / 2)
    qw = np.cos(phi / 2) * np.cos(theta / 2) * np.cos(psi / 2) + np.sin(phi / 2) * np.sin(theta / 2) * np.sin(psi / 2)

    return [qw, qx, qy, qz]

def random_center(min_val=-1.0, max_val=1.0, seed = 0):
    random.seed(seed)   
    return np.random.uniform(min_val, max_val, size=3)

In [41]:
# check the random radius and random_quaternion and the random_center:
print(random_radius(seed=123))

3.7858767423914466


In [9]:
from scipy.spatial.transform import Rotation as R
import numpy as np

def interpolate_poses(camera1, camera2, alpha):
    # Interpolate radius
    new_radius = (1 - alpha) * camera1.radius + alpha * camera2.radius
    
    # Interpolate center
    new_center = (1 - alpha) * camera1.center + alpha * camera2.center
    
    # Spherical Linear Interpolation (SLERP) for quaternions
    rot1 = camera1.rot
    rot2 = camera2.rot
    new_rot = Slerp(rot1, rot2, [alpha])[0]
    
    # Create and return new interpolated camera
    new_camera = OrbitCamera(W=camera1.W, H=camera1.H)
    new_camera.radius = new_radius
    new_camera.center = new_center
    new_camera.rot = new_rot
    
    return new_camera

# Initialize camera1 and camera2 with some random poses
camera1 = OrbitCamera(W=800, H=600)
camera2 = OrbitCamera(W=800, H=600)

camera1.radius = random_radius()
camera1.rot = R.from_quat(random_quaternion())
camera1.center = random_center()

camera2.radius = random_radius()
camera2.rot = R.from_quat(random_quaternion())
camera2.center = random_center()

# Interpolate between camera1 and camera2 poses
alpha = 0.5  # Interpolation parameter, varies between 0 and 1
interpolated_camera = interpolate_poses(camera1, camera2, alpha)


TypeError: __init__() takes exactly 3 positional arguments (4 given)

In [ ]:
import math


def calculate_focal_length(diagonal, camera_angle):
    focal_length = diagonal / (2 * math.tan(camera_angle / 2))
    return focal_length

# diagnal is the width and the height of the image 
def calculate_diagonal(width, height):
    diagonal = math.sqrt(width**2 + height**2)
    return diagonal

image_width = 512
image_height = 512



In [15]:
import numpy as np

# Original list
original_list = list(range(50))  # [0, 1, 2, ..., 49]

# Extend the list to a length of 101
repeat_factor = 101 // len(original_list) 
extended_list = np.repeat(original_list, repeat_factor)

print(extended_list)

inverse_list = extended_list[::-1]

#concate the list
extended_list = np.concatenate((extended_list, inverse_list))
extended_list = extended_list[:101]

print(extended_list)


[ 0  0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11
 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23
 24 24 25 25 26 26 27 27 28 28 29 29 30 30 31 31 32 32 33 33 34 34 35 35
 36 36 37 37 38 38 39 39 40 40 41 41 42 42 43 43 44 44 45 45 46 46 47 47
 48 48 49 49]
[ 0  0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11
 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23
 24 24 25 25 26 26 27 27 28 28 29 29 30 30 31 31 32 32 33 33 34 34 35 35
 36 36 37 37 38 38 39 39 40 40 41 41 42 42 43 43 44 44 45 45 46 46 47 47
 48 48 49 49 49]
